In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
import xgboost as xgb

In [ ]:
df = pd.read_csv("../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Train.csv")
test = pd.read_csv("../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Train.csv")
sub = pd.read_csv("../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Sample Submission.csv")
test.InvoiceDate = pd.to_datetime(test.InvoiceDate)
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
df

In [ ]:
test

In [ ]:
df.columns
numerical = ['InvoiceNo', 'StockCode', 'Description', 'Quantity',
       'CustomerID', 'Country', 'day', 'month', 'year']
num = ['InvoiceNo', 'StockCode', 'Description',
       'CustomerID']
# for n in num:
#     df[n] = np.power(df[n],0.5)
#     test[n] = np.power(test[n],0.5)
# test

In [ ]:
def date_time(x):
    x["Day"] = [i.day for i in x.InvoiceDate] 
    x["Month"] = [i.month for i in x.InvoiceDate] 
    x["Year"] = [i.year for i in x.InvoiceDate] 
    x["Minute"] = [i.minute for i in x.InvoiceDate] 
    return x
date_time(test)
date_time(df)

In [ ]:
df_final = df
test_final = test

In [ ]:
plt.figure(figsize=(40,20))
plt.plot(df.UnitPrice)

In [ ]:
df = df[df.UnitPrice < 35000]

In [ ]:
sns.distplot(df.InvoiceNo)

In [ ]:
sns.distplot(test.InvoiceNo)

In [ ]:
sns.distplot(df.StockCode)

In [ ]:
sns.distplot(test.StockCode)

In [ ]:
sns.distplot(df.Description)

In [ ]:
sns.distplot(test.StockCode)

In [ ]:
sns.distplot(df.Quantity)

In [ ]:
sns.distplot(test.Quantity)

In [ ]:
sns.distplot(df.CustomerID)

In [ ]:
sns.distplot(test.CustomerID)

In [ ]:
sns.countplot(df.Country)

In [ ]:
sns.countplot(test.Country)

In [ ]:
test.Quantity.value_counts()[0:8]

In [ ]:
sns.boxplot(df.UnitPrice)

In [ ]:
# sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [8, 5]})
# fig, ax = plt.subplots(2, 3, figsize=(20, 10))
# for variable, subplot in zip(numerical, ax.flatten()):
#     sns.histplot(df[variable],ax=subplot)
#     #sm.qqplot(data_points, line ='45',ax=subplot) 
#     for label in subplot.get_xticklabels():
#         label.set_rotation(90)
# fig.tight_layout() 

In [ ]:
# sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [8, 5]})
# fig, ax = plt.subplots(2, 3, figsize=(20, 10))
# for variable, subplot in zip(numerical, ax.flatten()):
#     sns.boxplot(df[variable],ax=subplot)
#     #sm.qqplot(data_points, line ='45',ax=subplot) 
#     for label in subplot.get_xticklabels():
#         label.set_rotation(90)
# fig.tight_layout() 

In [ ]:
# sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [8, 5]})
# fig, ax = plt.subplots(2, 3, figsize=(20, 10))
# for variable, subplot in zip(numerical, ax.flatten()):
#     sns.regplot(x=np.power(df[df.UnitPrice < 3000][variable],0.5),y= df[df.UnitPrice < 3000]['UnitPrice'],data=df[df.UnitPrice < 500],ax=subplot)
#     #sm.qqplot(data_points, line ='45',ax=subplot) 
#     for label in subplot.get_xticklabels():
#         label.set_rotation(90)
# fig.tight_layout() 

In [ ]:
sns.heatmap(df.corr())

In [ ]:
sns.heatmap(test.corr())

In [ ]:
df = df[(df.Quantity < 10000) | (df.Quantity > -20000)]

In [ ]:
df['Country_afe'] = df.Country.apply(lambda x: 1 if x == 35 else 0)
test['Country_afe'] = test.Country.apply(lambda x: 1 if x == 35 else 0)

In [ ]:
sns.barplot(x = df.Country,y = df.UnitPrice)

In [ ]:
def transform(data,feature,grouping):
    if grouping == "Month":
        one = feature + "_month_add_afe"
        two = feature + "_month_avg_afe"
        three = feature + "_month_count_afe"
        four = feature + "_month_median_afe"
    elif grouping == "Day":
        one = feature + "_day_add_afe"
        two = feature + "_day_avg_afe"
        three = feature + "_day_count_afe"
        four = feature + "_day_median_afe"
    elif grouping == "Minute":
        one = feature + "_minute_add_afe"
        two = feature + "_minute_avg_afe"
        three = feature + "_minute_count_afe"
        four = feature + "_minute_median_afe"
    else:
        one = feature + "_year_add_afe"
        two = feature + "_year_avg_afe"
        three = feature + "_year_count_afe"
        four = feature + "_year_median_afe"
        
    data[one] = data[grouping].map(data.groupby(grouping).sum()[feature].to_dict())
    data[two] = data[grouping].map(data.groupby(grouping).mean()[feature].to_dict())
    data[three] = data[grouping].map(data.groupby(grouping).count()[feature].to_dict())
    data[four] = data[grouping].map(data.groupby(grouping).median()[feature].to_dict())
#transform(df,"InvoiceNo",'month')

In [ ]:
#transform(test,"InvoiceNo",'month')
#without day only xg gives 5 and 6 xg max depth 5 with day 5 and 7 max depth of 4 xg
for month in ["Day","Month",'Year',"Minute"]:
    for att in ["InvoiceNo","StockCode","Description","Quantity","CustomerID","Country"]:
        transform(df,att,month)
        transform(test,att,month)
# for att in ["InvoiceNo","StockCode","Description","Quantity","CustomerID","Country"]:
#     for at in ["InvoiceNo","StockCode","Description","Quantity","CustomerID","Country"]:
#         if att != at:
#             one = att + "_add_" + at
#             two = att + "_sub_" + at
#             three = att + "_multiply_" + at
#             four = att + "_divide_" + at
#             df[one] = df[att] + df[at]
#             df[two] = df[att] - df[at]
#             df[three] = df[att] * df[at]
#             df[four] = df[att] / df[at]
#             test[one] = test[att] + test[at]
#             test[two] = test[att] - test[at]
#             test[three] = test[att] * test[at]
#             test[four] = test[att] / test[at]
            
df

In [ ]:
test

In [ ]:
corr = df.corr()['UnitPrice']
corr

In [ ]:
np.sort(corr)[:30]

In [ ]:
print(np.sort(corr)[-30:-5])
for i in np.sort(corr)[-30:-5]:
    print(np.where(corr == i))
#print(np.where(corr == i) for i in np.sort(corr)[-30:-5])   

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X_train,X_valid, y_train,y_valid = train_test_split(df[df.UnitPrice <3000].drop(["InvoiceDate","UnitPrice"],axis = 1),df[df.UnitPrice <3000].UnitPrice,test_size=0.1,random_state = 0)
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
knn = XGBRegressor(max_depth = 5, n_estimators = 300,random_state = 0,n_jobs = -1)
knn.fit(X_train,y_train)
print(np.sqrt(mean_squared_error(y_train,knn.predict(X_train))))
print(np.sqrt(mean_squared_error(y_valid,knn.predict(X_valid))))

In [ ]:
plt.plot(knn.feature_importances_)

In [ ]:
df.isnull().sum().value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(knn, max_num_features=50, height=0.5, ax=ax,importance_type='weight')
plt.show()

In [ ]:
results=pd.DataFrame()
results['column']= X_train.columns
results['importances'] = knn.feature_importances_
results.sort_values(by='importances',ascending=False,inplace=True)

results['column'][:10].values

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
model = Sequential()

# add the encoding layer.
model.add(layers.Dense(256, input_shape=(X_train.shape[1],)))

# add the output layer.
model.add(layers.Dense(64,))

model.add(layers.Dense(1))
# compile the model, you can use whatever optimizer.
model.compile(optimizer='adadelta', loss='mean_squared_error')
model.fit(X_train,y_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_valid, y_valid))